In [1]:
## DO NOT ERASE THIS. IMPORTANT TO CORRECTLY IMPORT MODULES
import sys

sys.path.append("../")
sys.executable

'/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/bin/python3.12'

In [2]:
import pandas as pd

pd.set_option("display.max_rows", 102)
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

from src.utils import (
    load_MovieLens,
    create_test_user,
    train_test_val_split,
    seed_everything,
    split_test_df,
    add_not_watched_movies,
    create_test_user_display_df,
    enrich_train_with_negatives,
)
from src.models import AutoInt
from src.metrics import reccomendation_report

## Constants

In [3]:
DATA_FOLDER = "../../data/ml-1m/"
RANDOM_STATE = 7

In [4]:
seed_everything(RANDOM_STATE)

## Data

In [5]:
df_users, df_movies, df_ratings = load_MovieLens(DATA_FOLDER)

### Creating a test_user

In [6]:
df_users, df_ratings, new_user_id = create_test_user(
    df_users, df_ratings, [6, 16, 3192, 1461, 827, 887, 593]
)
print("Test user watch list:")
display(df_ratings[df_ratings.userId == new_user_id].merge(df_movies, on="movieId"))

Test user watch list:


,userId,movieId,rating,timestamp,name,genre
0,6040,6,5,0,Sabrina (1995),Comedy|Romance
1,6040,16,5,1,Sense and Sensibility (1995),Drama|Romance
2,6040,3192,5,2,Singles (1992),Comedy|Drama|Romance
3,6040,1461,5,3,Love and Other Catastrophes (1996),Romance
4,6040,827,5,4,Emma (1996),Comedy|Drama|Romance
5,6040,887,5,5,Singin' in the Rain (1952),Musical|Romance
6,6040,593,5,6,Pretty Woman (1990),Comedy|Romance


### Train-test split
Methodology: Last user interaction is a test item. The rest is train. Validation part is 20% of test.

In [7]:
df_train, df_test, df_val = train_test_val_split(df_ratings, df_movies, RANDOM_STATE)

Enriching test:   0%|          | 0/4832 [00:00<?, ?it/s]

Enriching val:   0%|          | 0/1209 [00:00<?, ?it/s]

### Enriching train with negatives

In [8]:
seed_everything(RANDOM_STATE)
df_train = enrich_train_with_negatives(df_train, df_movies)

  0%|          | 0/994175 [00:00<?, ?it/s]

### Loading additional data

In [9]:
# cos_dist = computeCosineSimilarities(df_train, "userId", "movieId", df_movies["movieId"].nunique())

# with open("../../data/cos_dist.pt", "wb") as f:
#     torch.save(cos_dist, f)

with open("../../data/cos_dist.pt", "rb") as f:
    cos_dist = torch.load(f)
popularity = torch.bincount(
    torch.tensor(df_train.movieId.values), minlength=df_movies["movieId"].nunique()
)
popularity = popularity / torch.max(popularity)

### Building ordinal encoded features and normalizing continious features

In [10]:
ord_user = OrdinalEncoder()
user_cat = pd.DataFrame(
    ord_user.fit_transform(df_users[["gender", "occupation"]]),
    columns=ord_user.feature_names_in_,
).assign(userId=df_users["userId"])

ord_movie = OrdinalEncoder()
movie_cat = pd.DataFrame(
    ord_movie.fit_transform(df_movies[["genre"]]), columns=ord_movie.feature_names_in_
).assign(movieId=df_movies["movieId"])

ss_user = StandardScaler()
user_num = pd.DataFrame(
    ss_user.fit_transform(df_users[["age"]]), columns=ss_user.feature_names_in_
).assign(userId=df_users["userId"])

df_train = (
    df_train[["userId", "movieId", "label"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
    .rename(columns={"label": "action"})
)
df_val = (
    df_val[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
)
df_test = (
    df_test[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
)

## AutoInt Best

Best hyper parametres combination after tuning:
Trial 2 finished with value: 0.6986755132675171 and parameters: {

'num_attn_blocks': 1, 

'n_mlp_layers': 7, 

'mlp_layers_dim': 496.0, 

'mlp_kwargs_dropout': False, 

'mlp_kwargs_batchnorm': True, 

'mlp_kwargs_dropout_rate': 0.6824616853932067, 

'lr': 0.004633256565142509, 

'n_epochs': 11.0

}

In [11]:
seed_everything(RANDOM_STATE)
autoint = AutoInt(
    task="classification",
    target=["action"],
    learning_rate=0.0046,
    head="LinearHead",
    max_epochs=11,
    num_attn_blocks=1,
    layers="496-496-496-496-496-496-496",
)

2024-05-14 11:46:37,482 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

In [12]:
%%time
autoint.fit(df_train)

Epoch 9/10 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389/389 0:00:41 • 0:00:00 9.56it/s  v_num: 42.000 train_loss: 0.255   
                                                                                 valid_loss: 0.266 valid_accuracy: 
                                                                                 0.884 train_accuracy: 0.890       
Validation ━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━━━ 21/98   0:00:01 • 0:00:06 14.69it/s

`Trainer.fit` stopped: `max_epochs=11` reached.


2024-05-14 11:54:59,705 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2024-05-14 11:54:59,709 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

CPU times: user 15 s, sys: 12 s, total: 27 s
Wall time: 8min 22s


In [13]:
df_test["pred"] = autoint.predict(df_test)
pred, target, pred_items = split_test_df(df_test, "userId", "movieId", "pred", "action")
reccomendation_report(pred, target, pred_items, cos_dist, popularity, k=15)

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)


{'Hit rate @ 15': tensor(0.7463),
 'NDCG @ 15': tensor(0.3973),
 'Diversity (ILD)': tensor(0.1139),
 'Novelty (EPC)': tensor(0.5662)}

## Test user recommendations

In [14]:
df_test_user = add_not_watched_movies(new_user_id, df_test, df_train, df_movies)
df_test_user = (
    df_test_user[["userId", "movieId", "action"]]
    .merge(user_cat, on="userId")
    .merge(user_num, on="userId")
    .merge(movie_cat, on="movieId")
)
df_test_user["pred"] = autoint.predict(df_test_user)

/Users/kristina/Desktop/University/COURSE_WORK/RecSys_thesis/recsysvenv/lib/python3.12/site-packages/pytorch_tabular/categorical_encoders.py:67: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_encoded[col].fillna(self._imputed, inplace=True)


In [15]:
display(
    create_test_user_display_df(
        df_test_user[["userId", "movieId", "pred"]], df_movies, "pred"
    )
)

,userId,movieId,name,genre,pred
0,6040,47,Pocahontas (1995),Animation|Children's|Musical|Romance,0.982717
1,6040,699,"Truth About Cats & Dogs, The (1996)",Comedy|Romance,0.970754
2,6040,2031,Splash (1984),Comedy|Fantasy|Romance,0.960241
3,6040,1526,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical,0.945909
4,6040,2014,"Muppet Christmas Carol, The (1992)",Children's|Musical,0.930079
5,6040,778,"Nutty Professor, The (1996)",Comedy|Fantasy|Romance|Sci-Fi,0.920984
6,6040,1249,Arsenic and Old Lace (1944),Comedy|Mystery|Thriller,0.916872
7,6040,903,Sabrina (1954),Comedy|Romance,0.916737
8,6040,360,"Lion King, The (1994)",Animation|Children's|Musical,0.906528
9,6040,2327,Shakespeare in Love (1998),Comedy|Romance,0.905723
